In [2]:
try:
    import IPython
except:
    %pip install IPython
    import IPython 
from IPython.display import display, IFrame, HTML, Javascript
from IPython.core.display import display, HTML

HTML("""<link rel="stylesheet" type="text/css" href="src/css/notebook.css"/>""")

# Rijksmuseum

The Rijksmuseum provides an API to query data. 

 - To start using the data and images, you first need to obtain an API key by registering for a Rijksstudio account. 
 - You will be given a key instantly upon request, which you can find at the advanced settings of your Rijksstudio account.
 
 #### Further Reading
 - [Rijksmuseum](https://www.rijksmuseum.nl/nl)
 - [Rijksmuseum API](https://data.rijksmuseum.nl/object-metadata/api/)

In [6]:
import requests
import json
import os
try:
    import pandas as pd
except:
    %pip install pandas
    import pandas as pd
    
fileNGA = 'data/nga/input/nga_ruskin.csv'
keyfile = "data/ruskin/input/keyrijks.txt"
key = open(keyfile, mode='r', encoding='utf-8-sig').read()
url = "https://www.rijksmuseum.nl/api/nl/collection?key=" + key + "&involvedMaker=John+Ruskin"
rijksdir = "data/rijks/json/"
outputdir = "data/ruskin/output/json/"
baseURI = "http://www.rijksmuseum.nl/nl/collectie/"


response = requests.get(url)
data = response.json()

for artwork in data["artObjects"]:

    if artwork["hasImage"] == False:
        continue
    if artwork["principalOrFirstMaker"] != "John Ruskin":
        continue
    text_file = open( rijksdir + artwork["id"] + ".json", "wt")
    text_file.write(json.dumps(artwork, indent=2))
    text_file.close()
    
    

In [7]:
mapp = {
    
    "id":"id",
    "accession_number":"objectNumber",
    "classification" : "",
    "title": "title",
    "alt_title": "longTitle",
    "notes": "",
    "date_created":"",
    "date_created_earliest": "",
    "date_created_latest": "",
    "created_period":"",
    "created_dynasty":"",
    "created_inscriptions":"",
    "created_notes": "",
    "creator":"principalOrFirstMaker",
    "physical_medium": "",
    "physical_style": "",
    "physical_technique": "",
    "physical_description": "",
    "physical_dimensions": "",
    "created_provenance": "" ,
    "credit_line": "",
    "collection" : "",
    "classification": "",
    "current_status" : "",
    "current_owner": "",
    "image_url":"",
    "homepage" : ""
}

display(pd.DataFrame(mapp, index=[0]).T)


,0
id,id
accession_number,objectNumber
classification,
title,title
alt_title,longTitle
notes,
date_created,
date_created_earliest,
date_created_latest,
created_period,


In [8]:
import re
try:
    import cromulent 
except:
    %pip install cromulent
    import cromulent
from cromulent.model import factory

from lib import linkedart as la

# list to hold file names for use with jsonld visualisation dropdown
selectOptions = []
selectOptions = [('Please select an artwork', '')]


def createObjProp(obj,mapp):
    objProp = {}
    csv_keys = list(obj.keys())
    for key in csv_keys:
        for prop in mapp:
            if key == mapp[prop]:
               
                if prop == "alt_title":
                    alt_title = obj[key]
                    # parse alt_title for dates
                    
                    years = re.findall('(\d{4})', alt_title)
                    if years[0]:
                        objProp["date_created_earliest"] = years[0]
                    if years[1]:
                        objProp["date_created_latest"] = years[1]
                        objProp["date_created"] = years[0] + "-" + years[1]
                    else:
                        objProp["date_created"] = years[0]

                else:
                    objProp[prop] = obj[key]
                    
    objProp["current_owner"] = {"name":"Rijksmuseum",
                                "location":"Amsterdam, Netherlands",
                                "type": "http://vocab.getty.edu/aat/300312281" ,
                                "type_label": "Museum"}
    return objProp   


file_list=os.listdir(rijksdir)
for file in file_list:
    l
        objProp=createObjProp(obj,mapp)
        
        objProp["image_url"] = obj["webImage"]["url"]
        objProp["creator"] = [{"id": baseURI + "creator/" + obj["id"], "name": obj["principalOrFirstMaker"], "role": "Artist"}]
        
        objProp["classification"] = "Painting"
        objProp["homepage"] = obj["links"]["web"]
        
        if objProp["creator"] != "":
          
            id = str(objProp["id"])
            object_uri = baseURI + id
    
            # create obj description
            objLA = la.createObjDesc(objProp,la.objTypes,object_uri)
            filename = id + ".json"
            selectOptions.append( ( objProp["title"] + " (" + filename + ")" , filename))
            # write to file 
            text_file = open(outputdir + filename, "wt")
            n = text_file.write(factory.toString(objLA, compact=False))
            text_file.close()
            

In [9]:
try:
    import ipywidgets
except:
    %pip install ipywidgets
    import ipywidgets

from ipywidgets import Layout, FileUpload 
from IPython.display import display, IFrame, HTML, Image

import os

try:
    import json
except:
    %pip install json
    import json 
    
from IPython.core.display import Javascript        
    
def dropdown_eventhandler(change):
    with open('./src/js/visld.js', 'r') as _jscript:
        code = _jscript.read() + "var file = '" + outputdir + change.new + "';var selector = '#visnga';visjsonld(file, selector); "
        display(Javascript(code))
    
        with open( outputdir + "/" + change.new) as json_file:
    
                artwork = json.load(json_file)
                if ("representation" in artwork):
                    image = artwork["representation"][0]["id"]
                    display(Javascript("document.getElementById('artworknga').src = '" + image   +   "';"))
                else:
                    display(Javascript("document.getElementById('artworknga').src = '';"))
                    

selectObject = ipywidgets.Dropdown(options=selectOptions)
selectObject.observe(dropdown_eventhandler, names='value')

display(selectObject)

Dropdown(options=(('Please select an artwork', ''), ('Bergkam aan het Meer van Genève (nl-RP-T-1987-19.json)',…

<div><img style="height:500px" id="artworknga" src=""/></div>

<div id='visnga' style='height:100%;width:6000px'></div>